In [1]:
#Resources
#https://machinelearningmastery.com/greedy-layer-wise-pretraining-tutorial/
#https://www.pyimagesearch.com/2018/12/31/keras-conv2d-and-convolutional-layers/
#https://scikit-learn.org/stable/auto_examples/classification/plot_digits_classification.html#sphx-glr-auto-examples-classification-plot-digits-classification-py
#https://towardsdatascience.com/a-simple-2d-cnn-for-mnist-digit-recognition-a998dbc1e79a

from sklearn.datasets import make_blobs
from tensorflow.python.keras.layers.normalization import BatchNormalization
from tensorflow.python.keras.layers.core import Activation
from tensorflow.python.keras.layers.core import Flatten
from tensorflow.python.keras.layers.core import Dropout
from tensorflow.python.keras.layers import Dense, Conv2D, MaxPooling2D
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.optimizers import SGD
from tensorflow.python.keras.utils import to_categorical
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.python.keras.optimizers import Adam
from os.path import isfile, join
from matplotlib import pyplot
from os import listdir, getcwd, chdir
import cv2
import glob
import numpy as np


# prepare the dataset
def prepare_data():
    #chdir(getcwd() +"\\PycharmProjects\\UnsupervisedPre-Training\\training-20200217T024831Z-001\\training")
    #onlyfiles = [f for f in listdir(getcwd()) if isfile(join(getcwd(), f))]
    #print(onlyfiles)
    path = getcwd() +"\\PycharmProjects\\UnsupervisedPre-Training\\training-20200217T024831Z-001\\training\\*.*"
    data = []
    for file in glob.glob(path):
        #print(file)
        img = cv2.imread(file)
        np.resize(img, (661, 570, 3))
        data.append (img)
    data0 = np.array(data)
    #data = data0.reshape(223,661,70,3)
    data1 = data[0:55]
    data2 = data[55:111]
    data3 = data[111:167]
    data4 = data[167:223]
    print(len(data1))
    print(len(data2))
    print(len(data3))
    print(len(data4))
    #img = load_img("FashionNova-Heels.jpg")  # this is a PIL image
    #x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
    #x = x.reshape((1,) + x.shape)
    return data1, data2, data3, data4


# define, fit and evaluate the base autoencoder
def base_autoencoder(train, test):
    chanDim = 1
    # define model
    model = Sequential()
    #print(train.shape)
    print(train[0].shape)
    model.add(Conv2D(32, (5, 5), activation="relu", input_shape=(661, 570, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Dropout(0.25))
    
    # softmax classifier
    model.add(Dense(2))
    model.add(Activation("softmax"))
    print("[INFO] compiling model...")
    opt = Adam(lr=1e-4, decay=1e-4/12)
    #model = StridedNet.build(width=96, height=96, depth=3, classes=len(lb.classes_), reg=l2(0.0005))
    model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
    print(model.summary())
    train_mse = model.evaluate(train, train, verbose=0)
    test_mse = model.evaluate(test, test, verbose=0)
    print('> reconstruction error train=%.3f, test=%.3f' % (train_mse, test_mse))
    return model


# evaluate the autoencoder as a classifier
def evaluate_autoencoder_as_classifier(model, trainX, trainy, testX, testy):
    # remember the current output layer
    output_layer = model.layers[-1]
    # remove the output layer
    model.pop()
    # mark all remaining layers as non-trainable
    for layer in model.layers:
        layer.trainable = False
    # fully-connected layer
    model.add(Flatten())
    model.add(Dense(512, kernel_initializer="he_normal"))
    model.add(Activation("relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    # evaluate model
    _, train_acc = model.evaluate(trainX, trainy, verbose=0)
    _, test_acc = model.evaluate(testX, testy, verbose=0)
    # put the model back together
    model.pop()
    model.add(output_layer)
    print("[INFO] compiling model...")
    opt = Adam(lr=1e-4, decay=1e-4 / args["epochs"])
    #model = StridedNet.build(width=96, height=96, depth=3, classes=len(lb.classes_), reg=l2(0.0005))
    model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])
    return train_acc, test_acc


# add one new layer and re-train only the new layer
def add_layer_to_autoencoder(model, trainX, testX):
    # remember the current output layer
    output_layer = model.layers[-1]
    # remove the output layer
    model.pop()
    # mark all remaining layers as non-trainable
    for layer in model.layers:
        layer.trainable = False
    # add a new hidden layer
    model.add(Conv2D(32, (5, 5), activation="relu", input_shape=(661, 570, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Dropout(0.25))
    # re-add the output layer
    model.add(output_layer)
    # fit model
    # evaluate reconstruction loss
    print(model.summary())
    train_mse = model.evaluate(trainX, trainX, verbose=0)
    test_mse = model.evaluate(testX, testX, verbose=0)
    print('> reconstruction error train=%.3f, test=%.3f' % (train_mse, test_mse))

def main():
    #prepare data
    train1, train2, test1, test2 = prepare_data()
    #print(test)
    # get the base autoencoder
    model = base_autoencoder(train1, test1)
    # evaluate the base model
    scores = dict()
    train_acc, test_acc = evaluate_autoencoder_as_classifier(model, train1, train2, test1, test2)
    print('> classifier accuracy layers=%d, train=%.3f, test=%.3f' % (len(model.layers), train_acc, test_acc))
    scores[len(model.layers)] = (train_acc, test_acc)
    # add layers and evaluate the updated model
    n_layers = 5
    for _ in range(n_layers):
        # add layer
        add_layer_to_autoencoder(model, trainX, testX)
        # evaluate model
        train_acc, test_acc = evaluate_autoencoder_as_classifier(model, trainX, trainy, testX, testy)
        print('> classifier accuracy layers=%d, train=%.3f, test=%.3f' % (len(model.layers), train_acc, test_acc))
        # store scores for plotting
        scores[len(model.layers)] = (train_acc, test_acc)
    # plot number of added layers vs accuracy
    keys = list(scores.keys())
    pyplot.plot(keys, [scores[k][0] for k in keys], label='train', marker='.')
    pyplot.plot(keys, [scores[k][1] for k in keys], label='test', marker='.')
    pyplot.legend()
    pyplot.savefig(os.getcwd() + "\\PycharmProjects\\UnsupervisedPre-Training\\test.png")
    pyplot.show()

main()

55
56
56
56
(661, 570, 3)
[INFO] compiling model...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 657, 566, 32)      2432      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 328, 283, 32)      0         
_________________________________________________________________
batch_normalization_v2 (Batc (None, 328, 283, 32)      1312      
_________________________________________________________________
dropout (Dropout)            (None, 328, 283, 32)      0         
_________________________________________________________________
dense (Dense)                (None, 328, 283, 2)       66        
_________________________________________________________________
activation (Activation)      (None, 328, 283, 2)       0         
Total params: 3,810
Trainable params: 3,154
Non-trainable params: 656


ValueError: Error when checking input: expected conv2d_input to have 4 dimensions, but got array with shape (661, 570, 3)